In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_39/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Checkpoint Evaluación de modelos e importancia de features

## Dataset

**Introducción**

La Organización Mundial de la Salud ha estimado que 12 millones de muertes ocurren en todo el mundo, cada año debido a enfermedades del corazón. La mitad de las muertes en los Estados Unidos y otros países desarrollados se deben a enfermedades cardiovasculares. El pronóstico temprano de las enfermedades cardiovasculares puede ayudar a tomar decisiones sobre los cambios en el estilo de vida en pacientes de alto riesgo y, a su vez, reducir las complicaciones. Esta investigación tiene la intención de identificar los factores más relevantes / de riesgo de enfermedad cardíaca, así como predecir el riesgo general mediante distintos modelos de clasificación

**Fuente**

El conjunto de datos está disponible públicamente en el sitio web de Kaggle (https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset), y proviene de un estudio cardiovascular en curso en residentes de la ciudad de Framingham, Massachusetts. El objetivo de la clasificación es predecir si el paciente tiene riesgo de enfermedad coronaria (CHD) en los próximos 10 años. El conjunto de datos proporciona la información del paciente. Incluye más de 4000 registros y 15 atributos.

**Variables**

Cada atributo es un factor de riesgo potencial. Hay factores de riesgo demográficos, conductuales y médicos.

Demográficos:

• Sex: male or female(Nominal)

• Age: Age of the patient;(Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous)
Behavioral

• Current Smoker: whether or not the patient is a current smoker (Nominal)

• Cigs Per Day: the number of cigarettes that the person smoked on average in one day.(can be considered continuous as one can have any number of cigarettes, even half a cigarette.)

Médicos (histórico):

• BP Meds: whether or not the patient was on blood pressure medication (Nominal)

• Prevalent Stroke: whether or not the patient had previously had a stroke (Nominal)

• Prevalent Hyp: whether or not the patient was hypertensive (Nominal)

• Diabetes: whether or not the patient had diabetes (Nominal)

Médicos (actual):

• Tot Chol: total cholesterol level (Continuous)

• Sys BP: systolic blood pressure (Continuous)

• Dia BP: diastolic blood pressure (Continuous)

• BMI: Body Mass Index (Continuous)

• Heart Rate: heart rate (Continuous - In medical research, variables such as heart rate though in fact discrete, yet are considered continuous because of large number of possible values.)

• Glucose: glucose level (Continuous)

Variable a predecir (target):

• 10 year risk of coronary heart disease CHD (binary: “1”, means “Yes”, “0” means “No”)



## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report


## Ejercicio 1 - Preparación de datos

1.1) Leamos los datos del archivo datasets_222487_478477_framingham.csv

1.2) ¿Qué porcentaje de registros hay en cada una de las categorías target?

1.3) ¿El dataset tiene datos faltantes?

1.4) Usemos `dropna` para eliminar los registros con valores faltantes, y volvamos a calcular el porcentaje de registros hay en cada una de las categorías target 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html


In [3]:
data_raw = pd.read_csv('../Data/datasets_222487_478477_framingham.csv')
data_raw.shape

(4238, 16)

In [4]:
data_raw.columns

Index(['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD'],
      dtype='object')

Veamos qué procentaje de registros hay en cada una de las categorías target

In [5]:
data_raw.TenYearCHD.value_counts() / data_raw.shape[0]

0    0.848042
1    0.151958
Name: TenYearCHD, dtype: float64

In [6]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4238 non-null   int64  
 1   age              4238 non-null   int64  
 2   education        4133 non-null   float64
 3   currentSmoker    4238 non-null   int64  
 4   cigsPerDay       4209 non-null   float64
 5   BPMeds           4185 non-null   float64
 6   prevalentStroke  4238 non-null   int64  
 7   prevalentHyp     4238 non-null   int64  
 8   diabetes         4238 non-null   int64  
 9   totChol          4188 non-null   float64
 10  sysBP            4238 non-null   float64
 11  diaBP            4238 non-null   float64
 12  BMI              4219 non-null   float64
 13  heartRate        4237 non-null   float64
 14  glucose          3850 non-null   float64
 15  TenYearCHD       4238 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 529.9 KB


In [7]:
data = data_raw.dropna()
data.shape

(3656, 16)

In [8]:
data.TenYearCHD.value_counts() / data.shape[0]

0    0.847648
1    0.152352
Name: TenYearCHD, dtype: float64

Vemos que en este caso, eliminando los registros que tienen algun valor nulo no cambiamos la proporción de registros en cada una de las categorías target

In [9]:
data.dtypes

male                 int64
age                  int64
education          float64
currentSmoker        int64
cigsPerDay         float64
BPMeds             float64
prevalentStroke      int64
prevalentHyp         int64
diabetes             int64
totChol            float64
sysBP              float64
diaBP              float64
BMI                float64
heartRate          float64
glucose            float64
TenYearCHD           int64
dtype: object

In [10]:
data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


## Ejercicio 2 - Train Test Split + StandardScaler

Construir los conjuntos de entranamiento y test y usando StandardScaler normalizar las features

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [11]:
X=data.drop(["TenYearCHD"],axis=1)
y=data["TenYearCHD"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=717)
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

## Ejercicio 3 - Opcional (pero recomendado)

Escribir una función que reciba como parámetros 

* la instancia de un modelo
* X_train, los registros con los valores de las features predictoras en el dataset de entrenamiento;
* y_train, los registros con el valor del target en el dataset de entrenamiento;
* X_test, los registros con los valores de las features predictoras en el dataset de testing;
* y_test, los registros con el valor del target en el dataset de testing;
* gridSearch_params, los parámetros para usar en grid search para la instancia del modelo
* gridSearch_bagging_params, los parámetros para usar en grid search para bagging

La función debe entrenar 

* El modelo que recibe como parámetro usando X_train, y_train

* Grid Search Cross Validation KFold: `cv_KFold = KFold(n_splits=3, shuffle=True, random_state=371)`
    

* Grid Search CV Stratified KFold: `StratifiedKFold(n_splits=3, shuffle=True, random_state=371)`

* Bagging

* Bagging Grid Search Stratified Cross Validation: usando como base el mejor estimador devuelto en Grid Search CV Stratified KFold y `StratifiedKFold(n_splits=3, shuffle=True, random_state=371)`

Esta función debe devolver alguna estructura de datos, por ejemplo un diccionario, donde después podamos consultar el score en test de cada uno de los modelos, y los valores de media y desvío del score en los entrenamientos con grid search.


In [12]:
def evaluate_model(model_instance, X_train, y_train, X_test, y_test, gridSearch_params, gridSearch_bagging_params):
    
    # entreno el modelo default
    model_instance.fit(X_train, y_train)
    
    # calculo el score sobre los datos de test
    score_default_test = model_instance.score(X_test, y_test)
    
    # calculo la matriz de confusión
    predictions_default = model_instance.predict(X_test)
    confusion_matrix_default = metrics.confusion_matrix(y_test, predictions_default)
    
    ###################################################
    
    # gridSearch KFold:    
    cv_KFold = KFold(n_splits=3, shuffle=True, random_state=371)
    grid_search_CV_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_KFold)    
    grid_search_CV_KFold_model.fit(X_train, y_train)        
    scores_KFold = cross_val_score(model_instance, X_train, y_train, cv=cv_KFold, n_jobs=-1)
    mean_score_grid_search_CV_KFold_model = scores_KFold.mean()
    std_score_grid_search_CV_KFold_model = scores_KFold.std()
        
    score_grid_search_CV_KFold_model = grid_search_CV_KFold_model.best_score_
    params_grid_search_CV_KFold_model = grid_search_CV_KFold_model.best_params_
    
    score_grid_search_CV_KFold_model_test = grid_search_CV_KFold_model.score(X_test, y_test)
    predictions_grid_search_CV_KFold_model = grid_search_CV_KFold_model.predict(X_test)    
    confusion_matrix_grid_search_CV_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_KFold_model)

    ###################################################
    
    # gridSearch Stratified KFold:    
    cv_Stratified_KFold =StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
    grid_search_CV_Stratified_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)    
    grid_search_CV_Stratified_KFold_model.fit(X_train, y_train)        
    scores_Stratified_KFold = cross_val_score(model_instance, X_train, y_train, cv=cv_Stratified_KFold, n_jobs=-1)
    mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
    std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    
    
    score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_score_
    params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_params_
    
    score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model.score(X_test, y_test)
    predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.predict(X_test)
    confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

    ###################################################
    
    if gridSearch_bagging_params:

        # bagging

        bagging_model_default = BaggingClassifier(base_estimator = model_instance)
        bagging_model_default.fit(X_train, y_train)
        score_bagging_model_default_test =  bagging_model_default.score(X_test, y_test)
        
        predictions_bagging_model_default = bagging_model_default.predict(X_test)
        confusion_matrix_bagging_model_default = metrics.confusion_matrix(y_test, predictions_bagging_model_default)    

        ###################################################

        # bagging Stratified KFold cross validation usando de base el mejor modelo de gridsearch estratificado
        base_estimator_stratified_grid_search = grid_search_CV_Stratified_KFold_model.best_estimator_
        cv_Stratified_KFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
        grid_search_bagging_model = GridSearchCV(BaggingClassifier(base_estimator = base_estimator_stratified_grid_search),
                               gridSearch_bagging_params, n_jobs=-1, cv = cv_Stratified_KFold)

        grid_search_bagging_model.fit(X_train, y_train)
        
        score_grid_search_bagging_model_test = grid_search_bagging_model.score(X_test, y_test)
        predictions_grid_search_bagging_model = grid_search_bagging_model.predict(X_test)
        confusion_matrix_grid_search_bagging_model = metrics.confusion_matrix(y_test, predictions_grid_search_bagging_model)
        scores_bagging_Stratified_KFold = cross_val_score(BaggingClassifier(base_estimator = base_estimator_stratified_grid_search),
                                                          X_train, y_train, cv=cv_Stratified_KFold, n_jobs=-1)
        mean_score_bagging_grid_search_CV_Stratified_KFold_model = scores_bagging_Stratified_KFold.mean()
        std_score_bagging_grid_search_CV_Stratified_KFold_model = scores_bagging_Stratified_KFold.std()    
        
        best_score_bagging_grid_search_CV_Stratified_KFold_model = grid_search_bagging_model.best_score_
        best_params_bagging_grid_search_CV_Stratified_KFold_model = grid_search_bagging_model.best_params_
    
    else:
        
        score_bagging_model_default_test = None
        confusion_matrix_bagging_model_default = None
        mean_score_bagging_grid_search_CV_Stratified_KFold_model = None
        std_score_bagging_grid_search_CV_Stratified_KFold_model = None
        confusion_matrix_grid_search_bagging_model = None
        best_score_bagging_grid_search_CV_Stratified_KFold_model = None
        score_grid_search_bagging_model_test = None
        best_params_bagging_grid_search_CV_Stratified_KFold_model = None

                                                                                                                                     
    ###################################################
    
    
    # armo un diccionario con todos los valores de performance que calculé para los modelos
    result = {
        'default': {
            'score': score_default_test,
            'confusion_matrix': confusion_matrix_default            
        },
        'cv_kfold': {
            'mean_score_grid_search': mean_score_grid_search_CV_KFold_model,
            'std_score_grid_search': std_score_grid_search_CV_KFold_model,
            'best_score_grid_search': score_grid_search_CV_KFold_model,
            'score': score_grid_search_CV_KFold_model_test,
            'confusion_matrix': confusion_matrix_grid_search_CV_KFold_model           
        },
        'cv_stratified_kfold': {
            'mean_score_grid_search': mean_score_grid_search_CV_Stratified_KFold_model,
            'std_score_grid_search': std_score_grid_search_CV_Stratified_KFold_model,
            'best_score_grid_search': score_grid_search_CV_Stratified_KFold_model,
            'score': score_grid_search_CV_Stratified_KFold_model_test,
            'confusion_matrix': confusion_matrix_grid_search_CV_Stratified_KFold_model           
        },
        'bagging': {
            'score': score_bagging_model_default_test,
            'confusion_matrix': confusion_matrix_bagging_model_default           
        },
        'bagging_cv_stratified_kfold': {
            'mean_score_grid_search': mean_score_bagging_grid_search_CV_Stratified_KFold_model,
            'std_score_grid_search': std_score_bagging_grid_search_CV_Stratified_KFold_model,
            'best_score_grid_search': best_params_bagging_grid_search_CV_Stratified_KFold_model,
            'score': score_grid_search_bagging_model_test,
            'confusion_matrix': confusion_matrix_grid_search_bagging_model           
        }
        
    }
    
    return result
        
#           'trained_model_instance': model_instance, 
#              'score_default': score_default_test,
#              'confusion_matrix_default': confusion_matrix_default,
#              'mean_score_grid_search_CV_KFold_model': mean_score_grid_search_CV_KFold_model, 
#              'std_score_grid_search_CV_KFold_model': std_score_grid_search_CV_KFold_model, 
#              'best_score_grid_search_CV_KFold_model': score_grid_search_CV_KFold_model,
#              'best_params_grid_search_CV_KFold_model': params_grid_search_CV_KFold_model,              
#              'score_grid_search_CV_KFold_model_test': score_grid_search_CV_KFold_model_test,
#              'confusion_matrix_grid_search_CV_KFold_model': confusion_matrix_grid_search_CV_KFold_model,
        
#              'mean_score_grid_search_CV_Stratified_KFold_model': mean_score_grid_search_CV_Stratified_KFold_model, 
#              'std_score_grid_search_CV_Stratified_KFold_model': std_score_grid_search_CV_Stratified_KFold_model, 
#              'best_score_grid_search_CV_Stratified_KFold_model': score_grid_search_CV_Stratified_KFold_model,
#              'best_params_grid_search_CV_Stratified_KFold_model': params_grid_search_CV_Stratified_KFold_model,              
#              'score_grid_search_CV_Stratified_KFold_model_test': score_grid_search_CV_Stratified_KFold_model_test,
#              'confusion_matrix_grid_search_CV_Stratified_KFold_model': confusion_matrix_grid_search_CV_Stratified_KFold_model,
        
#              'score_bagging_default': score_bagging_model_default_test,
#              'confusion_matrix_bagging_default': confusion_matrix_bagging_model_default,
        
#              'mean_score_bagging_grid_search_CV_Stratified_KFold_model': mean_score_bagging_grid_search_CV_Stratified_KFold_model, 
#              'std_score_bagging_grid_search_CV__StratifiedKFold_model': std_score_bagging_grid_search_CV_Stratified_KFold_model, 
#              'best_score_bagging_grid_search_CV_Stratified_KFold_model': best_score_bagging_grid_search_CV_Stratified_KFold_model,
#              'best_params_bagging_grid_search_CV_Stratified_KFold_model': best_params_bagging_grid_search_CV_Stratified_KFold_model,              
#              'score_bagging_grid_search_CV_Stratified_KFold_model_test': score_grid_search_bagging_model_test,
#              'confusion_matrix_bagging_grid_search_CV_Stratified_KFold_model': confusion_matrix_grid_search_bagging_model,

    
    
    
    
    

## Ejercicio 4

Entrenar un modelo de clasificación basado en árboles, usando bagging y grid search cross validation estratificado sobre los parámetros de bagging, y calcular el score en test de cada uno de los modelos, y los valores de media y desvío del score en los entrenamientos con grid search cross validation.

Los parámetros de GridSearchCV que vamos a probar son 

``params =  {'criterion': ['gini', 'entropy'],
          'splitter': ['best', 'random'],
          'max_depth': [None, 5, 10],
          'min_samples_split': [2, 5],
          'min_samples_leaf': [1, 2, 3]}``

Los parámetros de bagging que vamos a probar son

<code>
bagging_params = {'n_estimators': [10, 100],

                  'max_samples': [0.01, 1.0],
                  
                  'max_features': [0.3, 1.0],
                  
                  'bootstrap_features': [True, False]}
</code>

Si resolvieron el ejercicio 3, pueden usar esa función. 

Si no, pueden hacer cada uno de los entrenamientos en la misma forma que venimos haciendo en las prácticas anteriores

In [13]:
model_instance = DecisionTreeClassifier()
x_train = X_train_sc
x_test =  X_test_sc
gridSearch_params = {'criterion': ['gini', 'entropy'],
          'splitter': ['best', 'random'],
          'max_depth': [None, 5, 10],
          'min_samples_split': [2, 5],
          'min_samples_leaf': [1, 2, 3]}

gridSearch_bagging_params = {'n_estimators': [10, 100],
                  'max_samples': [0.01, 1.0],                  
                  'max_features': [0.3, 1.0],                  
                  'bootstrap_features': [True, False]}


In [14]:
#usando la función:

decision_tree_metrics = evaluate_model(model_instance, x_train, y_train, x_test, y_test, gridSearch_params, gridSearch_bagging_params)

decision_tree_metrics

{'default': {'score': 0.7584320875113947,
  'confusion_matrix': array([[788, 144],
         [121,  44]])},
 'cv_kfold': {'mean_score_grid_search': 0.7592809691285658,
  'std_score_grid_search': 0.018593790157671475,
  'best_score_grid_search': 0.8464243845252053,
  'score': 0.8477666362807658,
  'confusion_matrix': array([[923,   9],
         [158,   7]])},
 'cv_stratified_kfold': {'mean_score_grid_search': 0.7674872997264556,
  'std_score_grid_search': 0.007184359636909028,
  'best_score_grid_search': 0.8444704962876123,
  'score': 0.8495897903372835,
  'confusion_matrix': array([[923,   9],
         [156,   9]])},
 'bagging': {'score': 0.837739288969918,
  'confusion_matrix': array([[902,  30],
         [148,  17]])},
 'bagging_cv_stratified_kfold': {'mean_score_grid_search': 0.8436889409925752,
  'std_score_grid_search': 0.0033615964310444385,
  'best_score_grid_search': {'bootstrap_features': True,
   'max_features': 1.0,
   'max_samples': 1.0,
   'n_estimators': 10},
  'score': 0.

In [15]:
# en la forma clásica

model_instance.fit(x_train, y_train)

# calculo el score sobre los datos de test
score_default_test = model_instance.score(x_test, y_test)

# calculo la matriz de confusión
predictions_default = model_instance.predict(X_test)
confusion_matrix_default = metrics.confusion_matrix(y_test, predictions_default)
    
###################################################
    
# gridSearch KFold:    
cv_KFold = KFold(n_splits=3, shuffle=True, random_state=371)
grid_search_CV_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_KFold)    
grid_search_CV_KFold_model.fit(x_train, y_train)        
scores_KFold = cross_val_score(model_instance, x_train, y_train, cv=cv_KFold, n_jobs=-1)
mean_score_grid_search_CV_KFold_model = scores_KFold.mean()
std_score_grid_search_CV_KFold_model = scores_KFold.std()

score_grid_search_CV_KFold_model = grid_search_CV_KFold_model.best_score_
params_grid_search_CV_KFold_model = grid_search_CV_KFold_model.best_params_

score_grid_search_CV_KFold_model_test = grid_search_CV_KFold_model.score(X_test, y_test)
predictions_grid_search_CV_KFold_model = grid_search_CV_KFold_model.predict(X_test)    
confusion_matrix_grid_search_CV_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_KFold_model)

###################################################
    
# gridSearch Stratified KFold:    
cv_Stratified_KFold =StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
grid_search_CV_Stratified_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)    
grid_search_CV_Stratified_KFold_model.fit(x_train, y_train)        
scores_Stratified_KFold = cross_val_score(model_instance, x_train, y_train, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_score_
params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model.score(X_test, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.predict(X_test)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

###################################################

# bagging

bagging_model_default = BaggingClassifier(base_estimator = model_instance)
bagging_model_default.fit(x_train, y_train)
score_bagging_model_default_test =  bagging_model_default.score(x_test, y_test)

predictions_bagging_model_default = bagging_model_default.predict(x_test)
confusion_matrix_bagging_model_default = metrics.confusion_matrix(y_test, predictions_bagging_model_default)    

###################################################

# bagging Stratified KFold cross validation usando de base el mejor modelo de gridsearch estratificado
base_estimator_stratified_grid_search = grid_search_CV_Stratified_KFold_model.best_estimator_
cv_Stratified_KFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
grid_search_bagging_model = GridSearchCV(BaggingClassifier(base_estimator = base_estimator_stratified_grid_search),
                       gridSearch_bagging_params, n_jobs=-1, cv = cv_Stratified_KFold)

grid_search_bagging_model.fit(x_train, y_train)

score_grid_search_bagging_model_test = grid_search_bagging_model.score(x_test, y_test)
predictions_grid_search_bagging_model = grid_search_bagging_model.predict(x_test)
confusion_matrix_grid_search_bagging_model = metrics.confusion_matrix(y_test, predictions_grid_search_bagging_model)
scores_bagging_Stratified_KFold = cross_val_score(BaggingClassifier(base_estimator = base_estimator_stratified_grid_search),
                                                  x_train, y_train, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_bagging_grid_search_CV_Stratified_KFold_model = scores_bagging_Stratified_KFold.mean()
std_score_bagging_grid_search_CV_Stratified_KFold_model = scores_bagging_Stratified_KFold.std()    

best_score_bagging_grid_search_CV_Stratified_KFold_model = grid_search_bagging_model.best_score_
best_params_bagging_grid_search_CV_Stratified_KFold_model = grid_search_bagging_model.best_params_

## Ejercicio 5 _ Performance

En este ejercicio vamos a repasar dos métodos para evaluar la performance de un modelo.

* accuracy_score - Accuracy = (TP + TN)/(TP + TN + FP + FN)

* classification_report
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

Vamos a evaluarlas sobre el modelo de bagging Stratified KFold cross validation que usa de base el mejor modelo de decision tree resultado gridsearch estratificado `StratifiedKFold(n_splits=3, shuffle=True, random_state=371)`


In [16]:
# el mejor modelo de gridsearch estratificado:

model_instance = DecisionTreeClassifier()
cv_Stratified_KFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
grid_search_CV_Stratified_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)    
grid_search_CV_Stratified_KFold_model.fit(x_train, y_train)        

best_model = grid_search_CV_Stratified_KFold_model.best_estimator_
y_pred = best_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)  
print('accuracy:', accuracy)
classification_rep = classification_report(y_test, y_pred)
print('classification_report:', classification_rep)
    

accuracy: 0.8486782133090246
classification_report:               precision    recall  f1-score   support

           0       0.86      0.99      0.92       932
           1       0.48      0.06      0.11       165

    accuracy                           0.85      1097
   macro avg       0.67      0.52      0.51      1097
weighted avg       0.80      0.85      0.80      1097



## Ejercicio 6 - Feature importance

Veamos qué importancia tiene cada una de las features del modelo entrenado en el ejercicio anterior

In [17]:
feature_names = X.columns
feature_importance = best_model.feature_importances_
importance = pd.DataFrame({'feature_names': feature_names, 'feature_importance': feature_importance})
importance.sort_values(by = 'feature_importance', axis=0, ascending=False, inplace=True)
importance

,feature_names,feature_importance
1,age,0.269384
7,prevalentHyp,0.249093
0,male,0.124906
3,currentSmoker,0.076510
8,diabetes,0.067987
10,sysBP,0.060438
4,cigsPerDay,0.038741
9,totChol,0.038673
11,diaBP,0.035421
13,heartRate,0.014627


## Ejercicio 7 - Extra

Calculen el área bajo la curva como medida de performance de alguno de los clasificadores entrenados

https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py


## Referencias 

Cross Validation
https://scikit-learn.org/stable/modules/cross_validation.html

StratifiedKFold
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold

Grid Search
https://scikit-learn.org/stable/modules/grid_search.html#grid-search

ROC
https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py

BaggingClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
